In [99]:
import numpy as np
from STL10 import read_all_images, read_labels
import tensorflow as tf
import matplotlib.pyplot as plt
from skimage.feature import hog

In [100]:
 # General data
X_train_imgs = read_all_images("stl10_binary/train_X.bin")
y_train = read_labels("stl10_binary/train_y.bin")

X_test_imgs = read_all_images("stl10_binary/test_X.bin")
y_test = read_labels("stl10_binary/test_y.bin")

 # Raw pixel values
X_train_raw = X_train_imgs/ 255
X_test_raw  = X_test_imgs / 255

print("raw:", X_train_raw.shape, X_test_raw.shape)

 # HoG
X_train_hog = []
X_test_hog = []

for img in X_train_imgs:
    desc, hogImg = hog(img, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1), visualize=True, multichannel=True)
    X_train_hog.append(hogImg )


for img in X_test_imgs:
    desc, hogImg = hog(img, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1), visualize=True, multichannel=True)
    X_test_hog.append(hogImg)

X_train_hog = np.array(X_train_hog)
X_test_hog = np.array(X_test_hog)
    
X_train_hog_3d = np.zeros([X_train_hog.shape[0],X_train_hog.shape[1],X_train_hog.shape[2],1])
X_test_hog_3d = np.zeros([X_test_hog.shape[0],X_test_hog.shape[1],X_test_hog.shape[2],1])

X_train_hog_3d[:,:,:,0] = X_train_hog
X_test_hog_3d[:,:,:,0] = X_test_hog

X_train_hog_3d = X_train_hog_3d / X_train_hog_3d.max()
X_test_hog_3d = X_test_hog_3d / X_test_hog_3d.max()

print("hog:", X_train_hog.shape, X_test_hog.shape)


 # Labels

num_categories = 10
y_train = tf.keras.utils.to_categorical(y_train-1, num_categories)
y_test = tf.keras.utils.to_categorical(y_test-1, num_categories)



raw: (5000, 96, 96, 3) (8000, 96, 96, 3)
hog: (5000, 96, 96) (8000, 96, 96)


In [101]:
def trainTestModel(X_trn, y_trn, X_tst, y_tst, steps):
    if len(X_trn.shape) == 4:
        shape = ( X_trn.shape[1], X_trn.shape[2], X_trn.shape[3] )
    elif len(X_trn.shape) == 3:
        shape = ( X_trn.shape[1], X_trn.shape[2], 1)
    else:
        return print("X_train shape must be 4 or 2 dimensions")
        
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=shape),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    #model.summary()
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

    history = model.fit(X_trn, y_trn, epochs=steps, verbose=1, validation_data=(X_tst, y_tst))
    #test_loss, test_acc = model.evaluate(X_tst,  y_tst, verbose=1, steps=steps)
    return history


In [102]:
steps = 4

In [103]:
 # Raw
trainTestModel(X_train_raw, y_train, X_test_raw, y_test, steps)

Epoch 1/4
157/157 [==============================] - 67s 426ms/step - loss: 4.4672 - accuracy: 0.3184 - val_loss: 1.7076 - val_accuracy: 0.3794
Epoch 2/4
157/157 [==============================] - 75s 479ms/step - loss: 1.1130 - accuracy: 0.6436 - val_loss: 1.4391 - val_accuracy: 0.4972
Epoch 3/4
157/157 [==============================] - 68s 431ms/step - loss: 0.4863 - accuracy: 0.8714 - val_loss: 2.1127 - val_accuracy: 0.4420
Epoch 4/4
157/157 [==============================] - 77s 493ms/step - loss: 0.1641 - accuracy: 0.9594 - val_loss: 4.2260 - val_accuracy: 0.3451


In [104]:
 # HoG
trainTestModel(X_train_hog_3d, y_train, X_test_hog_3d, y_test, steps)

Epoch 1/4
157/157 [==============================] - 64s 408ms/step - loss: 1.5610 - accuracy: 0.4414 - val_loss: 1.5889 - val_accuracy: 0.4849
Epoch 2/4
157/157 [==============================] - 62s 397ms/step - loss: 1.1893 - accuracy: 0.5752 - val_loss: 1.7775 - val_accuracy: 0.4350
Epoch 3/4
157/157 [==============================] - 62s 395ms/step - loss: 1.0592 - accuracy: 0.6338 - val_loss: 1.3941 - val_accuracy: 0.5274
Epoch 4/4
157/157 [==============================] - 59s 379ms/step - loss: 0.9584 - accuracy: 0.6708 - val_loss: 1.4444 - val_accuracy: 0.5479
